In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from scipy.stats import qmc

In [2]:
def make_param_list(param_data, out_file):
    
    df_list = []
    for parameter in param_data.data_vars:
        dims = [dim for dim in param_data[parameter].dims]
        attrs = param_data[parameter].attrs
        df = pd.DataFrame({'parameter_name': [parameter],
                          'coord': [dims],
                          })
        for attr in attrs:
            df[attr] = attrs[attr]
        df_list.append(df)
    param_df = pd.concat(df_list)
    param_df.to_csv(out_file)

    return param_df

In [3]:
def get_percentage_change(param_change):

    return float(param_change.replace('percent', ''))

In [4]:
def get_param_value(change_str, default_value, param_list_file,
                    parameter, type):
    
    if change_str == 'pft':
        pft_values = pd.read_excel(param_list_file,
                                   parameter.replace("fates_", ""))
        param_value = pft_values[type].values
    
    elif 'percent' in change_str:
        percent_change = get_percentage_change(change_str)
        if type == 'param_min':
            param_value = default_value - np.abs(default_value*(percent_change/100.0))
        else:
            param_value = default_value + np.abs(default_value*(percent_change/100.0))
    else:
        numeric_value = float(change_str)
        if default_value.size == 1:
            param_value = numeric_value
        else:
            param_value = np.full_like(default_value, numeric_value)

    if default_value.size == 1:
        if default_value == -999.9:
            param_value = -999.9
    else:
        param_value[default_value == -999.9] = -999.9

    return param_value

In [5]:
def make_oaat_parameter_file(param_list, default_value, default_param_data,
                             param_list_file, parameter, type, param_dir,
                             param_prefix, suffix):
    
    change_str = str(param_list[type].values[0])
    param_value = get_param_value(change_str, default_value,
                                  param_list_file, parameter, type)
    ds = default_param_data.copy(deep=False)
    ds[parameter].values = param_value
    ds.to_netcdf(os.path.join(param_dir,
                             f"{param_prefix}_{suffix}.nc"))

In [6]:
def create_oaat_param_ensemble(param_list, param_list_file, default_param_data, param_dir, param_prefix):
    
    parameters = np.unique(param_list.fates_parameter_name)
    
    oaat_key_list = []
    ens = 1
    for parameter in parameters:
        
        sub = param_list[param_list.fates_parameter_name == parameter]
        default_value = default_param_data[parameter].values
        
        make_oaat_parameter_file(sub, default_value, default_param_data,
                                 param_list_file, parameter, 'param_min',
                                 param_dir, param_prefix, str(ens).zfill(3))
        df = pd.DataFrame({'ensemble': [str(ens).zfill(3)],
                           'type': ['min'],
                           'parameter_name': [parameter],
                          })
        oaat_key_list.append(df)
        ens += 1
        
        make_oaat_parameter_file(sub, default_value, default_param_data,
                                 param_list_file, parameter, 'param_max', param_dir,
                                 param_prefix, str(ens).zfill(3))
        df = pd.DataFrame({'ensemble': [str(ens).zfill(3)],
                   'type': ['max'],
                    'parameter_name': [parameter],
                  })
        oaat_key_list.append(df)
        ens += 1
    oaat_key = pd.concat(oaat_key_list)
    oaat_key.to_csv(os.path.join(param_dir, 'fates_oaat_key.csv'))

    file_names = [f"{param_prefix}_{ens}" for ens in oaat_key.ensemble.values]
    with open(os.path.join(param_dir, 'FATES_OAAT.txt'), 'w') as f:
        for line in file_names:
            f.write(f"{line}\n")

In [7]:
def get_lh_values(value, param_list, param_list_file, param, default_value):
    
    sub = param_list[param_list.fates_parameter_name == param]
    change_str_min = str(sub['param_min'].values[0])
    change_str_max = str(sub['param_max'].values[0])

    min_value = get_param_value(change_str_min, default_value,
                                  param_list_file, param, 'param_min')
    max_value = get_param_value(change_str_max, default_value,
                                  param_list_file, param, 'param_max')

    new_value = unnormalize(value, min_value, max_value)

    return new_value

In [8]:
def create_lh_param_ensemble(params, num_sample, default_param_data,
                             param_list, param_list_file, keep_pfts=[]):

    num_params = len(params)
    sampler = qmc.LatinHypercube(d=num_params)
    lh_sample = sampler.random(n=num_sample)
    
    for i, sample in enumerate(lh_sample):
        
        ds = default_param_data.copy(deep=False)
        
        for j, value in enumerate(sample):
            if params[j] == 'smpsc_delta':
                if 'fates_nonhydro_smpso' in params:
                    default_smpso = default_param_data['fates_nonhydro_smpso'].values
                    k = params.index('fates_nonhydro_smpso')
                    smpso_value = get_lh_values(sample[k], param_list, param_list_file, 'fates_nonhydro_smpso',
                                                default_smpso)
                else:
                    smpso_value = default_param_data['fates_nonhydro_smpso'].values
                
                smpsc_delta = get_lh_values(value, param_list, param_list_file, params[j], np.array([189000]))
                new_value = smpso_value - smpsc_delta
                if len(keep_pfts) > 0:
                    default_value = default_param_data['fates_nonhydro_smpsc'].values
                    for pft in keep_pfts:
                        new_value[pft-1] = default_value[pft-1]
                ds['fates_nonhydro_smpsc'].values = new_value
            
            else:
                default_value = default_param_data[params[j]].values
                new_value = get_lh_values(value, param_list, param_list_file, params[j], default_value)

                dims = [dim for dim in default_param_data[params[j]].dims]
                if len(keep_pfts) > 0 and 'fates_pft' in dims:
                    for pft in keep_pfts:
                        if len(dims) == 2:
                            new_value[0, pft-1] = default_value[0, pft-1]
                        else:
                            new_value[pft-1] = default_value[pft-1]
                ds[params[j]].values = new_value
        ds.to_netcdf(os.path.join(param_dir, f"{param_prefix}_{str(i+1).zfill(3)}.nc"))
    
    lh_key = pd.DataFrame(lh_sample)
    lh_key.columns = params
    lh_key['ensemble'] = [f"FATES_LH_{str(ens).zfill(3)}" for ens in np.arange(1, num_sample+1)]
    
    return lh_key

In [9]:
def normalize(value, min_value, max_value):
    return (value - min_value)/(max_value - min_value)

In [10]:
def unnormalize(value, min_value, max_value):
    return (max_value - min_value)*value + min_value

In [11]:
param_dir = '/glade/work/afoster/FATES_calibration/parameter_files'
fates_param_name = "fates_params_default_sci.1.78.3._api.36.1.0.nc"
param_list_name = "param_list_sci.1.78.3_api.36.1.0.xls"

In [12]:
default_param_file = os.path.join(param_dir, fates_param_name)
param_list_file = os.path.join(param_dir, param_list_name)

default_param_data = xr.open_dataset(default_param_file, decode_cf=False)
param_list = pd.read_excel(param_list_file, sheet_name=0)

In [ ]:
# param_dir = '/glade/work/afoster/FATES_calibration/parameter_files/fates_param_oaat'
# param_prefix = 'FATES_OAAT'
# create_oaat_param_ensemble(param_list, param_list_file, default_param_data,
#                            param_dir, param_prefix)

In [16]:
param_dir = '/glade/work/afoster/FATES_calibration/parameter_files/fates_param_lh_leaf'
param_prefix = 'FATES_LH'

In [14]:
num_sample = 500
# params_to_cal = ["fates_leaf_vcmax25top", "fates_rad_leaf_clumping_index",
#                  "smpsc_delta", "fates_nonhydro_smpso", "fates_leafn_vert_scaler_coeff2",
#                  "fates_leaf_stomatal_intercept", "fates_leaf_stomatal_slope_medlyn",
#                  "fates_leafn_vert_scaler_coeff1", "fates_maintresp_leaf_atkin2017_baserate",
#                  "fates_turb_leaf_diameter"]

params_to_cal = ["fates_leaf_jmaxha", "fates_leaf_jmaxhd", "fates_leaf_jmaxse",
                 "fates_leaf_stomatal_intercept", "fates_leaf_stomatal_slope_medlyn",
                 "fates_leaf_theta_cj_c3", "fates_leaf_vcmax25top", "fates_leaf_vcmaxha",
                 "fates_leaf_vcmaxhd", "fates_leaf_vcmaxse"]
len(params_to_cal)

10

In [18]:
lh_key = create_lh_param_ensemble(params_to_cal, num_sample, default_param_data,
                                  param_list, param_list_file)

In [19]:
lh_key.to_csv(os.path.join(param_dir, 'fates_lh_key_leaf.csv'), index=False)

In [20]:
ensembles = lh_key.ensemble.values
with open(os.path.join(param_dir, 'FATES_LH.txt'), 'w') as f:
    for ensemble in ensembles:
        f.write(f"{ensemble}\n")